# Подключаем гугл диск

In [1]:
!pip install efficientnet_pytorch

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


# Распаковка данных

In [3]:
# !unzip -n "/content/gdrive/MyDrive/ups_2022_circle/circle_data.zip" -d "/content/gdrive/MyDrive/ups_2022_circle/data"

In [4]:
# ! cp -a '/content/gdrive/MyDrive/ups_2022_circle/data/ups_2022_circle/train_1/circle/.' '/content/gdrive/MyDrive/ups_2022_circle/data/train/circle/' 

# Подгружаем зависимости

In [5]:
! pip3 install torch torchvision torchaudio

# На CUDA будет побыстрее

In [6]:
import torch
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [7]:
!pip install Pillow==5.3.0
import PIL
print(PIL.PILLOW_VERSION)

5.3.0


In [8]:
!nvidia-smi
import torch
torch.cuda.is_available()

Sat Nov 20 06:09:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

True

In [9]:
import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

# в sklearn не все гладко, чтобы в colab удобно выводить картинки 
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [10]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")

# Подгружаем данные с трех разных датасетов 

In [11]:
TRAIN_DIR_1 = Path('/content/gdrive/MyDrive/agrocode/data_3')

train_val_files = sorted(list(TRAIN_DIR_1.rglob('*.jpg')) + list(TRAIN_DIR_1.rglob('*.JPG')))

In [12]:
len(train_val_files)

1565

In [13]:
!pip install efficientnet_lite_pytorch

# install the pretrained model file you're interested in, e.g., lite0

!pip install efficientnet_lite0_pytorch_model

In [14]:
# from efficientnet_lite_pytorch import EfficientNet
# from efficientnet_lite0_pytorch_model import EfficientnetLite0ModelFile
# weights_path = EfficientnetLite0ModelFile.get_model_file_path()

# lite0_model = EfficientNet.from_pretrained('efficientnet-lite0', weights_path = weights_path )


In [15]:
# tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
# numm = 0
# for path in tqdm(train_val_files):
#   img = tfms(Image.open(path)).unsqueeze(0)
#   img = lite0_model.extract_features(img)
#   numm += 1 
#   torch.save(img, '/content/gdrive/MyDrive/agrocode/efficient_tensors/{x}/{y}.pt'.format(x = path.parent.name, y = numm))

# # train_val_files = torch.cat(imgs, dim=0)


In [16]:
FEATURE_DIR = Path('/content/gdrive/MyDrive/agrocode/efficient_tensors/')
train_val_files = sorted(list(FEATURE_DIR.rglob('*.pt')))
train_val_features = torch.cat([torch.load(i) for i in train_val_files], dim = 0)
train_val_features = train_val_features.reshape(len(train_val_files), -1)

In [32]:
from sklearn.model_selection import train_test_split

train_val_labels = [1 if path.parent.name == 'desease' else 0 for path in train_val_files]

X_train, X_test, Y_train, Y_test = train_test_split(train_val_features, train_val_labels, test_size=0.2, \
                                          stratify=train_val_labels)

In [33]:
X_train = X_train.detach().numpy()
X_test = X_test.detach().numpy()

In [34]:
type(X_train[0][0])

numpy.float32

In [35]:
import xgboost as xgb
import pandas as pd
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20
model = xgb.train(param, D_train, steps)

In [36]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])


print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.8386677075201665
Recall = 0.7576972576972577
Accuracy = 0.8402555910543131


# Посмотрим на графики лосса

In [ ]:
loss, acc, val_loss, val_acc = zip(*history)

In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(loss, label="train_loss")
plt.plot(val_loss, label="val_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(loss, label="train_loss")
plt.plot(val_loss, label="val_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()


# Сохраним веса получившейся модели


In [ ]:
torch.save(simple_cnn.state_dict(), "/content/gdrive/MyDrive/ups_2022_circle/weights.pth")